# Make extended versions for FRET, etc.

### Imports

In [1]:
%load_ext lab_black
# Python standard library
from glob import glob
import os
import socket
import sys

# 3rd party library imports
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas
# Notebook magic
# save plots in the notebook
%matplotlib inline
# reloads modules automatically before executing cells
%load_ext autoreload
%autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

running in directory: /mnt/projects/crispy_shifty/projects/crispy_shifties
running on node: gpu25


### Set working directory to the root of the crispy_shifty repo

In [2]:
os.chdir("/projects/crispy_shifty")

### Start on some from Flo

In [3]:
hinge = "/mnt/home/flop/switch/5thround/extend/cs221_Y_n1c1.pdb"
basename = "cs_221AB_n1c1"
num_repeats_extended = 1

sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.protocols.cleaning import design_after_repeat_propagation

simulation_name = "11_make_extensions"
output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")
os.makedirs(output_path, exist_ok=True)
pdbs_path = os.path.join(output_path, "pdbs")
os.makedirs(pdbs_path, exist_ok=True)

pposes = design_after_repeat_propagation(
    None,
    pdb_path=hinge,
    models="all",
    num_repeats_extended=num_repeats_extended,
    num_sequences=8,
)

for i, p in enumerate(pposes):
    p.pose.dump_pdb(os.path.join(pdbs_path, f"{basename}_{str(i).zfill(4)}.pdb"))

/projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 39 in file /mnt/home/flop/switch/5thround/extend/cs221_Y_n1c1.pdb. Best match rsd_type:  GLU
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't find pose atom for file-residue 39 atom 1H   (trying to store temperature in PDBInfo)
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't find pose atom for file-residue 39 atom 2H   (trying to store temperature in PDBInfo)
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't find pose atom for file-residue 39 atom 3H   (trying to store temperature in PDBInfo)
Scores may be absent or incorrectly formatted
0.09 min: Redesigning caps with MPNN

----------------------------------------
pssm_jsonl is NOT loaded
----------------------------------------
omit_AA_jsonl is NOT loaded
----------------------------------------
bias_AA_jsonl is NOT loaded
----------------------------------------
tied_positions_jsonl is NOT loaded
--------------------

In [ ]:
hinge = "/mnt/home/flop/switch/5thround/extend/cs221_Y_n2c2.pdb"
basename = "cs_221AB_n2c2"
num_repeats_extended = 2

sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.protocols.cleaning import design_after_repeat_propagation

simulation_name = "11_make_extensions"
output_path = os.path.join(os.getcwd(), f"projects/crispy_shifties/{simulation_name}")
os.makedirs(output_path, exist_ok=True)
pdbs_path = os.path.join(output_path, "pdbs")
os.makedirs(pdbs_path, exist_ok=True)

pposes = design_after_repeat_propagation(
    None,
    pdb_path=hinge,
    max_recycles=5,
    models="all",
    num_repeats_extended=num_repeats_extended,
    num_sequences=48,
)

for i, p in enumerate(pposes):
    p.pose.dump_pdb(os.path.join(pdbs_path, f"{basename}_{str(i).zfill(4)}.pdb"))

/projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 77 in file /mnt/home/flop/switch/5thround/extend/cs221_Y_n2c2.pdb. Best match rsd_type:  GLU
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't find pose atom for file-residue 77 atom 1H   (trying to store temperature in PDBInfo)
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't find pose atom for file-residue 77 atom 2H   (trying to store temperature in PDBInfo)
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] can't find pose atom for file-residue 77 atom 3H   (trying to store temperature in PDBInfo)
Scores may be absent or incorrectly formatted
0.15 min: Redesigning caps with MPNN

----------------------------------------
pssm_jsonl is NOT loaded
----------------------------------------
omit_AA_jsonl is NOT loaded
----------------------------------------
bias_AA_jsonl is NOT loaded
----------------------------------------
tied_positions_jsonl is NOT loaded
--------------------

### Make new extensions

In [ ]:
def range_CA_align(pose1, pose2, start1, end1, start2, end2):
#modified from Adam's function (pose 1 mobile)
    pose1_residue_selection = range(start1,end1)
    pose2_residue_selection = range(start2,end2)
    
    assert len(pose1_residue_selection) == len(pose2_residue_selection)

    pose1_coordinates = pyrosetta.rosetta.utility.vector1_numeric_xyzVector_double_t()
    pose2_coordinates = pyrosetta.rosetta.utility.vector1_numeric_xyzVector_double_t()

    for pose1_residue_index, pose2_residue_index in zip(pose1_residue_selection, pose2_residue_selection):
        pose1_coordinates.append(pose1.residues[pose1_residue_index].xyz('CA'))
        pose2_coordinates.append(pose2.residues[pose2_residue_index].xyz('CA'))

    rotation_matrix = pyrosetta.rosetta.numeric.xyzMatrix_double_t()
    pose1_center = pyrosetta.rosetta.numeric.xyzVector_double_t()
    pose2_center = pyrosetta.rosetta.numeric.xyzVector_double_t()

    pyrosetta.rosetta.protocols.toolbox.superposition_transform(pose1_coordinates,
                                                                pose2_coordinates,
                                                                rotation_matrix,
                                                                pose1_center,
                                                                pose2_center)

    pyrosetta.rosetta.protocols.toolbox.apply_superposition_transform(pose1,
                                                                      rotation_matrix,
                                                                      pose1_center,
                                                                      pose2_center)
                     
def clash_check(pose):
    all_gly = pose.clone()
    true_sel = pyrosetta.rosetta.core.select.residue_selector.TrueResidueSelector()
    true_x = true_sel.apply(all_gly)
    pyrosetta.rosetta.protocols.toolbox.pose_manipulation.repack_these_residues(true_x, all_gly, sfxn, False, "G" )
    score = sfxn(all_gly)
    return score

def helix_dict(pose):
    #pose = pyrosetta.io.pose_from_pdb(filename)
    ss = pyrosetta.rosetta.core.scoring.dssp.Dssp(pose)
    hlx_dict = {}
    n = 1
    for i in range(1,len(pose.sequence())):
        if (ss.get_dssp_secstruct(i) == "H") & (ss.get_dssp_secstruct(i-1) !="H"):
            hlx_dict[n] = [i]
        if (ss.get_dssp_secstruct(i) == "H") & (ss.get_dssp_secstruct(i+1) !="H"):
            hlx_dict[n].append(i)
            n +=1
    return hlx_dict

def load_pose(name):
    if type(name) == str:
        pose = pyrosetta.io.pose_from_pdb(name)
    else:
        try:
            pose = name.clone()
        except:
            print("input must be a pose or a path to a pose")
    return pose


def swap_ab(name):
    pose_ab = load_pose(name)
    pose_ba = pose_ab.split_by_chain(2)
    pyrosetta.rosetta.core.pose.append_pose_to_pose(pose_ba,pose_ab.split_by_chain(1),1)
    return pose_ba
# swap_ab: normally assume that oligomer is chain A and heterodimer is chain B. If the other way round ,then do swap_ab

def extend_dhr(pose1,pose2, keep = "c", h1n = "last", h2n = "last", h1_c = 3, h2_c = 4):
    if type(pose1) == str:
        pose_n = pyrosetta.io.pose_from_pdb(pose1).split_by_chain(1)
    else:
        pose_n = pose1.split_by_chain(1)
    if type(pose2) == str:
        pose_c = pyrosetta.io.pose_from_pdb(pose2).split_by_chain(1)
    else:
        pose_c = pose2.split_by_chain(1)
    hdn = helix_dict(pose_n)
    hdc = helix_dict(pose_c)
    if (h1n == "last") or (h1n == -2):
        h1_n = list(hdn)[-2]
    else:
        h1_n = h1n
    if (h2n == "last") or (h2n == -1):
        h2_n = list(hdn)[-1]
    else:
        h2_n = h2n
    copy_c = pose_c.clone()
    newpose = pyrosetta.rosetta.core.pose.Pose()
    start_c = hdc[h1_c][0]
    end_c = hdc[h2_c][1]
    start_n = hdn[h1_n][0]
    end_n = hdn[h2_n][1]
    if (end_n - start_n) != (end_c - start_c):
        #print(end_n - start_n)
        #print(end_c - start_c)
        if keep == "n":
            start_c = end_c - (end_n - start_n)
        elif keep == "c":
            end_n = start_n + (end_c - start_c) 
        else:
            print("keep must be n or c")
        while (start_c < 1) or (start_n < 1):
            start_c +=1
            start_n +=1
    range_CA_align(copy_c, pose_n, start_c, end_c, start_n, end_n)
    if keep == "n":
        cut1 = end_n 
        cut2 = end_c         
    elif keep == "c":
        cut1 = start_n 
        cut2 = start_c 
    else:
        print("keep must be n or c")
    #print(cut1,cut2)
    for i in range(1,cut1):
        newpose.append_residue_by_bond(pose_n.residue(i))
    for i in range(cut2,len(pose_c.sequence())+1):
        newpose.append_residue_by_bond(copy_c.residue(i))
    offset = (len(newpose.sequence())-len(pose_c.sequence()))    
    
    selx = pyrosetta.rosetta.core.select.residue_selector.ResiduePDBInfoHasLabelSelector("IntX")
    sely = pyrosetta.rosetta.core.select.residue_selector.ResiduePDBInfoHasLabelSelector("IntY")
    int_n = pyrosetta.rosetta.core.select.residue_selector.OrResidueSelector(selx,sely).apply(pose_n)
    int_c = pyrosetta.rosetta.core.select.residue_selector.OrResidueSelector(selx,sely).apply(pose_c)
    for i in pyrosetta.rosetta.core.select.get_residues_from_subset(int_n):
        newpose.replace_residue(i,pose_n.residue(i),1)
    for i in pyrosetta.rosetta.core.select.get_residues_from_subset(int_c):
        newpose.replace_residue(i+offset,pose_c.residue(i),1)
    info = pyrosetta.rosetta.core.pose.PDBInfo(newpose,1)
    for label in ["IntX","IntY"]:
        sel = pyrosetta.rosetta.core.select.residue_selector.ResiduePDBInfoHasLabelSelector(label)
        int_n = sel.apply(pose_n)
        int_c = sel.apply(pose_c)
        for i in pyrosetta.rosetta.core.select.get_residues_from_subset(int_n):            
            info.add_reslabel(i,label)
        for i in pyrosetta.rosetta.core.select.get_residues_from_subset(int_c):            
            info.add_reslabel(i+offset,label)
        newpose.pdb_info(info)
    return newpose
    
    
def oligomer_align(oligomer,monomer,const_term = "n", mono_chains = [1]):
    oligo = load_pose(oligomer)
    mono_full = load_pose(monomer)
    if mono_chains == [1]:
        mono = mono_full
    else:
        mono = pyrosetta.rosetta.core.pose.Pose()
        for i in mono_chains:
            pyrosetta.rosetta.core.pose.append_pose_to_pose(mono,mono_full.split_by_chain(i),1)
    if const_term in ["n","N"]:
        start_m = 1
        end_m = len(oligo.chain_sequence(1))
    elif const_term in ["c","C"]:
        start_m = mono.size() - len(oligo.chain_sequence(1)) + 1
        end_m = mono.size()
    else:
        print("const_term must be n or c")
    newpose = pyrosetta.rosetta.core.pose.Pose()
    for i in range(oligo.num_chains()):
        start_o = 1 + (i * len(oligo.chain_sequence(1)))
        end_o = ((i+1) * len(oligo.chain_sequence(1)))
        range_CA_align(mono, oligo, start_m, end_m, start_o, end_o)
        if i == 0:
            newpose = mono.clone()
        if i >= 1:
            for j in range(1,mono.num_chains()+1):
                pyrosetta.rosetta.core.pose.append_pose_to_pose(newpose,mono.split_by_chain(j),1)
        
    return newpose

def swap_chains(name,order):
    pose_in = load_pose(name)
    pose_out = pyrosetta.rosetta.core.pose.Pose()
    for i in order:
        pyrosetta.rosetta.core.pose.append_pose_to_pose(pose_out,pose_in.split_by_chain(i),1)
    return pose_out


def extend_oligomer(oligomer,extension,term,keep,oligo_hlx1,oligo_hlx2,ext_hlx1,ext_hlx2):
    oligo = load_pose(oligomer)
    ext = load_pose(extension)
    oligo_sym = oligo.num_chains()
    ext_chains = range(1,ext.num_chains()+1)
    if term in ['N','n']:
        p1 = extend_dhr(extension,oligomer,keep,ext_hlx1,ext_hlx2,oligo_hlx1,oligo_hlx2)
        p2 = oligomer_align(oligomer,p1,"c",ext_chains)
    elif term in ['C','c']:
        p1 = extend_dhr(oligomer,extension,keep,oligo_hlx1,oligo_hlx2,ext_hlx1,ext_hlx2)
        p2 = oligomer_align(oligomer,p1,"n",[x-1+oligo_sym for x in ext_chains])
    else:
        print("term must be n or c")
    return p2

def extend_oligo_hinge(hinge,parent,nrep=2,crep=2):
    par = load_pose(parent).split_by_chain(1)
    hin = load_pose(hinge)
    nres = []
    cres = []
    offset = hin.split_by_chain(1).size() - par.size()
    for i in range(1,par.size()//2 - 10):
        if par.sequence(i,i) != hin.sequence(i,i):
            nres.append(i)
    for i in range(par.size()//2 + 10,par.size()):
        if par.sequence(i,i) != hin.sequence(i+offset,i+offset):
            cres.append(i)        
    print(nres)
    print(cres)
    p1 = extend_dhr(par,hin.split_by_chain(1),"n",5,6,1,2)
    offset_n = p1.split_by_chain(1).size() - hin.split_by_chain(1).size()
    print(offset_n,offset)
    p2 = oligomer_align(hin,p1,const_term = "c")
    p3 = extend_dhr(p2.split_by_chain(1),par,"c","last","last",3,4)
    offset_c = p3.split_by_chain(1).size() - p1.split_by_chain(1).size()
    for i in nres:
        p3.replace_residue(i+offset_n,hin.residue(i),1)
    for i in cres:
        p3.replace_residue(i+offset_n+offset,hin.residue(i+offset),1)
    p4 = oligomer_align(p2,p3,const_term = "n")
    return p4

def extend_mono_hinge(hinge,parent,nrep=2,crep=2):
    par = load_pose(parent).split_by_chain(1)
    hin = load_pose(hinge)
    nres = []
    cres = []
    offset = hin.split_by_chain(1).size() - par.size()
    for i in range(1,par.size()//2 - 10):
        if par.sequence(i,i) != hin.sequence(i,i):
            nres.append(i)
    for i in range(par.size()//2 + 10,par.size()):
        if par.sequence(i,i) != hin.sequence(i+offset,i+offset):
            cres.append(i)        
    print(nres)
    print(cres)
    if nrep ==2:
        p1 = extend_dhr(par,hin.split_by_chain(1),"c",7,8,3,4)
    elif nrep ==1: 
        p1 = extend_dhr(par,hin.split_by_chain(1),"c",5,6,3,4)
    offset_n = p1.split_by_chain(1).size() - hin.split_by_chain(1).size()
    print(offset_n,offset)
    p2 = p1
    if crep == 2:
        p3 = extend_dhr(p2.split_by_chain(1),par,"c","last","last",3,4)
    elif crep == 1:
        p3 = extend_dhr(p2.split_by_chain(1),par,"c","last","last",5,6)
    offset_c = p3.split_by_chain(1).size() - p1.split_by_chain(1).size()
    for i in nres:
        p3.replace_residue(i+offset_n,hin.residue(i),1)
    for i in cres:
        p3.replace_residue(i+offset_n+offset,hin.residue(i+offset),1)
    p4 = p3
    return p4

In [ ]:
p = extend_mono_hinge("in/cs_221X.pdb","in/DHR20.pdb",1,1)
p.dump_pdb("cs221_X_n1c1.pdb")
p = extend_mono_hinge("in/cs_221Y.pdb","in/DHR20.pdb",1,1)
p.dump_pdb("cs221_Y_n1c1.pdb")
p = extend_mono_hinge("in/cs_221X.pdb","in/DHR20.pdb")
p.dump_pdb("cs221_X_n2c2.pdb")
p = extend_mono_hinge("in/cs_221Y.pdb","in/DHR20.pdb")
p.dump_pdb("cs221_Y_n2c2.pdb")

In [ ]:
p = extend_dhr('in/DHR14a.pdb','in/J7X_G133P.pdb',"n", 3,4,1,2)
p.dump_pdb('J7X_P_extn1.pdb')
p = extend_dhr('in/DHR14a.pdb','in/J7Y_G133P.pdb',"n", 3,4,1,2)
p.dump_pdb('J7Y_P_extn1.pdb')

In [ ]:
p = extend_dhr('in/DHR14a.pdb','in/J7X_G133P.pdb',"n", 5,6,1,2)
p.dump_pdb('J7X_P_extn2.pdb')
p = extend_dhr('in/DHR14a.pdb','in/J7Y_G133P.pdb',"n", 5,6,1,2)
p.dump_pdb('J7Y_P_extn2.pdb')

In [ ]:
p = extend_dhr('in/DHR14b.pdb','in/J7X_G133P.pdb',"n", 3,4,1,2)
p.dump_pdb('J7X_P_extn1_dhr14x.pdb')
p = extend_dhr('in/DHR14b.pdb','in/J7Y_G133P.pdb',"n", 3,4,1,2)
p.dump_pdb('J7Y_P_extn1_dhr14x.pdb')
p = extend_dhr('in/DHR14b.pdb','in/J7X_G133P.pdb',"n", 5,6,1,2)
p.dump_pdb('J7X_P_extn2_dhr14x.pdb')
p = extend_dhr('in/DHR14b.pdb','in/J7Y_G133P.pdb',"n", 5,6,1,2)
p.dump_pdb('J7Y_P_extn2_dhr14x.pdb')

In [ ]:
p = extend_mono_hinge("in/cs201r1_X.pdb","in/DHR82.pdb",1,1)
p.dump_pdb("cs201_X_n1c1.pdb")
p = extend_mono_hinge("in/cs201r1_Y.pdb","in/DHR82.pdb",1,1)
p.dump_pdb("cs201_Y_n1c1.pdb")
p = extend_mono_hinge("in/cs201r1_X.pdb","in/DHR82.pdb")
p.dump_pdb("cs201_X_n2c2.pdb")
p = extend_mono_hinge("in/cs201r1_Y.pdb","in/DHR82.pdb")
p.dump_pdb("cs201_Y_n2c2.pdb")

In [ ]:
p = extend_mono_hinge("in/pair157_232_Y_0001.pdb","in/DHR82.pdb")
p.dump_pdb("FF74_Y_n2c2_new_unrel.pdb")

In [ ]:
p = extend_mono_hinge("in/pair157_232_X_0001.pdb","in/DHR82.pdb",1,1)
p.dump_pdb("FF74_X_n1c1.pdb")
p = extend_mono_hinge("in/pair157_232_Y_0001.pdb","in/DHR82.pdb",1,1)
p.dump_pdb("FF74_Y_n1c1.pdb")

In [ ]:
p = extend_hinge("in/FF86-0.pdb","in/DHR21.pdb")
p.dump_pdb("v3_FF86-0_n2c2.pdb")
p = extend_hinge("in/FF86-1.pdb","in/DHR21.pdb")
p.dump_pdb("v3_FF86-1_n2c2.pdb")
p = extend_hinge("in/FF86-2.pdb","in/DHR21.pdb")
p.dump_pdb("v3_FF86-2_n2c2.pdb")

In [ ]:
p = extend_oligo_hinge("in/FF81-0.pdb","in/DHR46.pdb")
p.dump_pdb("v3_FF81-0_n2c2.pdb")
p = extend_oligo_hinge("in/FF81-1.pdb","in/DHR46.pdb")
p.dump_pdb("v3_FF81-1_n2c2.pdb")

In [ ]:
helix_dict(load_pose("in/pair157_232_X_0001.pdb"))

In [ ]:
helix_dict(load_pose("/projects/crispy_shifty/data/pdbs/DHR20.pdb"))[1]

In [ ]:
hd = helix_dict(load_pose("in/DHR46.pdb"))
for i in list(hd):
    print(hd[i][1]-hd[i][0])

In [ ]:
hd = helix_dict(load_pose("in/FF81-1.pdb"))
for i in list(hd):
    print(hd[i][1]-hd[i][0])

In [ ]:
pyrosetta.rosetta.protocols.protein_interface_design.FavorNativeResidue?

In [ ]:
p = extend_oligo_hinge("in/FF81-0.pdb","in/DHR46.pdb")

In [ ]:
res = p.residue(4)

In [ ]:
res.